# <img src="uni-logo.png" alt="Logo UNI" width=150 hight=300 align="right">


<br><br><br>
<h1><font color="#7F000E" size=4>Minería de Datos (CC442)</font></h1>



<h1><font color="#7F000E" size=6>Teoría: Algoritmo del Ball-Tree </font></h1>

<br>
<div style="text-align: right">
<font color="#7F000E" size=3>Yuri Coicca, M.Sc.</font><br>
<font color="#7F000E" size=3>Facultad de Ciencias</font><br>
<font color="#7F000E" size=3>Ciencia de la Computación - UNI</font><br>
</div>

El **Ball-Tree** es una estructura de datos de particionado espacial diseñada para organizar puntos en un espacio de múltiples dimensiones. Se creó específicamente para superar la **"maldición de la dimensionalidad"** que afecta al KD-Tree.

A diferencia del KD-Tree, que divide el espacio con hiperplanos ortogonales (cajas), el Ball-Tree divide los puntos en **hiperesferas** (bolas).

---

### 1. ¿Por qué Ball-Tree en lugar de KD-Tree?
En dimensiones altas (muchas variables), el KD-Tree se vuelve ineficiente porque las "esquinas" de las cajas del hiperrectángulo dejan mucho espacio vacío, obligando a revisar casi todas las ramas. Las esferas del Ball-Tree envuelven los datos de forma más compacta, permitiendo una **poda (pruning)** de ramas mucho más agresiva.

---

### 2. Construcción del Ball-Tree
El algoritmo agrupa los datos de forma jerárquica y recursiva:

1.  **Seleccionar el centro:** Se eligen dos puntos muy alejados entre sí en el conjunto actual (a menudo usando el eje de mayor varianza). Estos serán los "centros" de los dos nodos hijos.
2.  **Asignación:** Cada punto del conjunto se asigna al centro más cercano.
3.  **Definir la bola (Ball):** Para cada grupo (hijo), se calcula:
    *   El **centroide** (la media de todos los puntos en ese grupo).
    *   El **radio** (la distancia desde el centroide al punto más lejano del grupo).
4.  **Recursión:** El proceso se repite para cada subgrupo hasta que se alcanza un número mínimo de puntos (hojas).

#### Pseudocódigo (Construcción):
```python
def construir_ball_tree(puntos):
    si len(puntos) < leaf_size:
        return NodoHoja(puntos)
    
    # Elegir dimensión con mayor dispersión o puntos más lejanos
    centroide = calcular_media(puntos)
    radio = calcular_distancia_maxima(centroide, puntos)
    
    izq_puntos, der_puntos = dividir_en_dos_clusters(puntos)
    
    return {
        'centro': centroide,
        'radio': radio,
        'izquierdo': construir_ball_tree(izq_puntos),
        'derecho': construir_ball_tree(der_puntos)
    }
```

---

### 3. Búsqueda del Vecino más Cercano
La búsqueda utiliza la desigualdad triangular para descartar grupos de puntos enteros.

1.  **Criterio de Poda:** Si tenemos un punto de consulta $Q$ y un "mejor vecino" actual con distancia $D_{best}$:
    *   Calculamos la distancia de $Q$ al centro de la bola actual ($D_{centro}$).
    *   Si $D_{centro} - Radio_{bola} > D_{best}$, significa que **ningún punto** dentro de esa bola puede estar más cerca que nuestro mejor vecino actual.
    *   **Resultado:** Se ignora (poda) toda esa rama del árbol.
2.  **Exploración:** Si no se puede podar, se baja por el árbol priorizando la bola cuyo centro esté más cerca de $Q$.

---

### 4. Comparativa: KD-Tree vs. Ball-Tree

| Característica | KD-Tree | Ball-Tree |
| :--- | :--- | :--- |
| **Forma de división** | Hiperrectángulos (ejes fijos) | Hiperesferas (cualquier dirección) |
| **Dimensiones** | Eficiente en bajas ($< 20$) | Eficiente en dimensiones medias/altas |
| **Costo de construcción** | Bajo (rápido) | Mayor (cálculo de distancias y radios) |
| **Precisión en poda** | Baja en alta dimensionalidad | Alta, más compacta |

---

### 5. Implementación en Python (Scikit-learn)
Es la forma más estándar de utilizarlo en Machine Learning:

```python
from sklearn.neighbors import BallTree
import numpy as np

# Datos de ejemplo (por ejemplo, 1000 puntos en 10 dimensiones)
X = np.random.random((1000, 10))

# 1. Construir el Ball-Tree
# leaf_size controla cuántos puntos hay en las hojas antes de parar
tree = BallTree(X, leaf_size=40)

# 2. Buscar los 3 vecinos más cercanos a un nuevo punto
punto_consulta = np.random.random((1, 10))
distancias, indices = tree.query(punto_consulta, k=3)

print(f"Índices de los vecinos: {indices}")
print(f"Distancias: {distancias}")
```

### Resumen de uso:
*   Usa **KD-Tree** si tus datos tienen pocas características (features) y quieres una construcción rápida.
*   Usa **Ball-Tree** si tienes un número moderado o alto de dimensiones donde el KD-Tree empieza a fallar (se vuelve tan lento como la búsqueda por fuerza bruta).